In [1]:
!pip install tensorflow==2.15 opencv-python pandas

  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
  Using cached opencv_python-4.10.0.82-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)
  Using cached pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl (24.5 MB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached 

In [12]:
!pip uninstall ml-dtypes --y
!pip install ml-dtypes
!pip install scikit-image

Found existing installation: ml-dtypes 0.4.0
Uninstalling ml-dtypes-0.4.0:
  Successfully uninstalled ml-dtypes-0.4.0
  Using cached ml_dtypes-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 60.4 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.1 MB/s eta 0:00:00
  Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
  Using cached imageio-2.34.1-py3-none-any.whl (313 kB)
  Using cached tifffile-2024.5.22-py3-none-any.whl (225 kB)

[notice] A new release of pip is available: 23.0

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2

from datetime import datetime

# from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


from keras.models import Sequential
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Embedding
from keras.optimizers import Adam

from keras.callbacks import TensorBoard

tf.__version__

'2.15.0'

In [3]:
%load_ext tensorboard

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# !unzip videos.zip
# df_train = pd.read_csv('/content/drive/MyDrive/TCC2/train_info.csv', index_col=0)
# df_train

!pip install mediapipe -q
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task
!pip install git+https://github.com/okankop/vidaug

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/okankop/vidaug to /tmp/pip-req-build-xwgtpuwc
  Running command git clone --filter=blob:none --quiet https://github.com/okankop/vidaug /tmp/pip-req-build-xwgtpuwc
  Resolved https://github.com/okankop/vidaug to commit 1c1ddf2640fe4a9171267d64ae5e3bd70c24d54a
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
folder_path = 'videos'
files = os.listdir(folder_path)
files.sort()
print(files[213])

tempo.mp4


In [5]:
pd.DataFrame({'palavra': list(map(lambda x: x.replace('.mp4', ''), files))}).to_csv('palavras.csv')

In [8]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_new_image(rgb_image, detection_result, stick=False):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)
  if stick:
    image_height, image_width, _ = rgb_image.shape
    annotated_image = np.zeros((image_height, image_width, 3)).astype('uint8')

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [9]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# from google.colab.patches import cv2_imshow

#inicializacao do framework mediapipe para fazer a extracao de caracteristicas do corpo humano
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)


max_frames = 90
def preprocess_image(image, target_size=None):
    """
    Pre processamento da imagem onde é responsavel pela transformação de
    RBG2Gray e redimensionar o tamanho da imagem
    """
    resized_image = cv2.resize(image, target_size) if target_size else image
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return resized_image


def read_videos(folder_path, target_size=None):
    """
    Função que é responsável pela leitura dos videos usando a lib do openCV e
    armazena cada frame do video em um array numpy junto com a extração de
    caracteristicas do corpo humano retornado do framework mediapipe e seu
    identificador
    """
    data = []
    mp_data = []
    labels = []
    files = os.listdir(folder_path)
    files.sort()

    if os.path.isfile('ariel_data.npy'):
      with open('ariel_data.npy', 'rb') as f:
          data = np.load(f)
          mp_data = np.load(f)
          labels = np.load(f)
    else:
      for idx, filename in enumerate(files):
          if filename.endswith('.mp4') or filename.endswith('.avi'):
              video_path = os.path.join(folder_path, filename)

              cap = cv2.VideoCapture(video_path)
              frames = []
              mp_frames = []
              while cap.isOpened():
                  ret, frame = cap.read()
                  if not ret:
                      break

                  processed_frame = preprocess_image(frame, target_size)
                  #frame pre processado
                  frames.append(processed_frame)
                  image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
                  detection_result = detector.detect(image)
                  #deteccao com o framework mediapipe se existe um corpo humano
                  #no frame
                  if detection_result.pose_world_landmarks:
                    annotated_image = draw_landmarks_new_image(cv2.cvtColor(image.numpy_view(), cv2.COLOR_BGR2RGB), detection_result)
                    annotated_image = preprocess_image(annotated_image, target_size)
                    mp_frames.append(annotated_image)
                  else:
                    mp_frames.append(processed_frame)


              cap.release()
              while len(frames) < max_frames:
                  frames.append(np.zeros_like(processed_frame))
              while len(mp_frames) < max_frames:
                  mp_frames.append(np.zeros_like(processed_frame))
              #normalizacao do frames contidos dentro da estrutura, foi pré
              #definido que cada video terá 90 frames, preenchido com zeros
              #caso nao atinja os 90 frames
              data.append(frames)
              mp_data.append(mp_frames)
              labels.append(idx)
      with open('ariel_data.npy', 'wb') as f:
        np.save(f, np.array(data))
        np.save(f, np.array(mp_data))
        np.save(f, np.array(labels))
        #salva e recupera o array numpy pois o processamentos de videos e lento

    return data, mp_data, labels

target_size = (128, 128)

data, mp_data, labels = read_videos(folder_path, target_size)

print("Shape dos dados:", data.shape)
print("Shape dos dados preprocces MediaPipe:", mp_data.shape)
print("Shape dos rótulos:", labels.shape)

I0000 00:00:1718223033.466950   10065 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1718223033.471999   10588 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.1.0-devel), renderer: Radeon RX 590 Series (radeonsi, polaris10, LLVM 17.0.6, DRM 3.42, 5.15.0-107-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1718223033.560122   10589 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718223033.630425   10604 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Shape dos dados: (228, 90, 128, 128)
Shape dos dados preprocces MediaPipe: (228, 90, 128, 128)
Shape dos rótulos: (228,)


In [10]:
X = data
# X_reshaped = X.reshape(-1, 128, 128, 1)

X_mp = mp_data
# X_mp = X.reshape(-1, 32, 32, 1)

y_train_repeated = np.repeat(labels, max_frames)
Y_reshaped = to_categorical(y_train_repeated)
Y = to_categorical(labels)

EPOCHS = 30
BATCH_SIZE = 512

#gerador de variações para o video ja que não possuimos um dataset amplo p treino
# train_generator = ImageDataGenerator(
#     rescale=1. / 255,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     brightness_range=(0.3, 1.0),
#     # horizontal_flip=True,
#     # vertical_flip=True,
#     validation_split=0.2).flow(X, Y, batch_size=BATCH_SIZE)


# train_generator_mp = ImageDataGenerator(
#     rescale=1. / 255,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     brightness_range=(0.3, 1.0),
#     # horizontal_flip=True,
#     # vertical_flip=True,
#     fill_mode='nearest',
#     validation_split=0.2).flow(X_mp, Y, batch_size=BATCH_SIZE)


# train_generator_reshaped = ImageDataGenerator(
#     rescale=1. / 255,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     brightness_range=(0.3, 1.0),
#     # horizontal_flip=True,
#     # vertical_flip=True,
#     fill_mode='nearest',
#     validation_split=0.2).flow(X_reshaped, Y_reshaped, batch_size=BATCH_SIZE)

# X.shape, train_generator, X_reshaped

/home/ballke/Documentos/TCC2/venv/lib/python3.10/site-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (228, 90, 128, 128) (128 channels).
  warnings.warn(


In [13]:
from vidaug import augmentors as va

sometimes = lambda aug: va.Sometimes(0.5, aug) # Used to apply augmentor with 50% probability
seq = va.Sequential([
    sometimes(va.RandomCrop(size=(128, 128))),
    # sometimes(va.RandomRotate(degrees=10)),
    # sometimes(va.VerticalFlip()),
    sometimes(va.HorizontalFlip()),
    sometimes(va.GaussianBlur(1.5))
])


X_variation = np.empty((0, 90, 128, 128, 1))
Y_variation = np.empty(0, dtype=int)

#os dados estao salvos pois gerar levam muito tempo e processamento para terminar
if os.path.isfile('ariel_data_variation.npy'):
  with open('ariel_data_variation.npy', 'rb') as f:
      X_variation = np.load(f)
      Y_variation = np.load(f)
else:
    for idx, video in enumerate(X):
      video = np.expand_dims(video, axis=-1)
      for _ in range(5):
          # 'video' should be either a list of images from type of numpy array or PIL images
          video_aug = np.array(seq(video))
          X_variation = np.concatenate((X_variation, [video_aug]), axis=0)
          Y_variation = np.concatenate((Y_variation, [idx]), axis=0)
          # for v in video_aug:
          #   cv2_imshow(v)
      print(f'variacoes de palavras processadas: {round((idx/len(files))*100, 2)}%')
    with open('ariel_data_variation.npy', 'wb') as f:
        np.save(f, X_variation)
        np.save(f, Y_variation)
X_variation.shape, Y_variation.shape

((1140, 90, 128, 128, 1), (1140,))

In [14]:

input_shape = (90, 128, 128, 1)
model = Sequential()
#rede resultante final, necessaria criar com filtros Conv2d em baixo numero para
# que consigamos compilar com baixo poder computacional
model.add(TimeDistributed(Conv2D(filters=8, kernel_size=3, activation='relu', padding='same'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(filters=4, kernel_size=3, activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=2)))

model.add(TimeDistributed(Flatten()))
# de maneira geral, colocar mais neuronios pra LSTM ao invés de camadas
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.5))
# tem que ser mesmo tamanho do vocabulário

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(labels), activation='softmax'))
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
#adicionado clipnorm para resolver problema de explosao de gradiente
model.compile(optimizer=optimizer, loss='categorical_crossentropy') # loss deve ser a crossentropy
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 90, 128, 128, 8)   80        
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 90, 128, 128, 8)   32        
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 90, 64, 64, 8)     0         
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 90, 64, 64, 16)    1168      
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 90, 64, 64, 16)    6

In [15]:
estop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.0001,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=1
)

Y_variation = to_categorical(Y_variation)
print(Y_variation.shape)
tensorboard = TensorBoard(log_dir="logs/train_cnnXrnn_baseline")
#treino com captura pro tensorboard, nao utilizei o estop pois parecia ter melhores resultados sem ele
#treino com o dataset apenas com preprocess
model.fit(X_variation, Y_variation, epochs=300,
          # validation_data=(X_variation, Y),
          callbacks=[tensorboard])

(1140, 228)
Epoch 1/300


2024-06-12 17:14:30.150739: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6723993600 exceeds 10% of free system memory.
2024-06-12 17:14:35.822265: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1509949440 exceeds 10% of free system memory.
2024-06-12 17:14:36.192727: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1509949440 exceeds 10% of free system memory.
2024-06-12 17:14:36.883903: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 754974720 exceeds 10% of free system memory.
2024-06-12 17:14:36.977037: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 754974720 exceeds 10% of free system memory.


36/36 [==============================] - 256s 7s/step - loss: 5.4656
Epoch 2/300
36/36 [==============================] - 252s 7s/step - loss: 5.4282
Epoch 3/300
36/36 [==============================] - 253s 7s/step - loss: 5.4152
Epoch 4/300
36/36 [==============================] - 253s 7s/step - loss: 5.3478
Epoch 5/300
36/36 [==============================] - 254s 7s/step - loss: 5.2033
Epoch 6/300
36/36 [==============================] - 255s 7s/step - loss: 5.0589
Epoch 7/300
36/36 [==============================] - 255s 7s/step - loss: 4.9323
Epoch 8/300
36/36 [==============================] - 255s 7s/step - loss: 4.8992
Epoch 9/300
36/36 [==============================] - 255s 7s/step - loss: 4.7548
Epoch 10/300
36/36 [==============================] - 255s 7s/step - loss: 4.6661
Epoch 11/300
36/36 [==============================] - 255s 7s/step - loss: 4.5899
Epoch 12/300
36/36 [==============================] - 255s 7s/step - loss: 4.5638
Epoch 13/300
36/36 [=================

IndexError: list index out of range

In [16]:
model.save('ariel-variationtrain.keras')

In [18]:
print('teste com dataset original')
cont = 0
for idx, test in enumerate(data):
  # x_test = test.reshape(-1, 32, 32, 1)
  if files[np.argmax(model.predict(np.array([test]), verbose=0))] == files[idx]:
    cont += 1
    print(files[idx])

print(f"o modelo teve {(cont/len(files))*100}% de acertos")

teste com dataset original
abril.mp4
acesso.mp4
acima.mp4
acordo.mp4
agora.mp4
agosto.mp4
ainda.mp4
alta.mp4
altura.mp4
ambos.mp4
americano.mp4
and.mp4
ano.mp4
anos.mp4
anterior.mp4
antes.mp4
apenas.mp4
apresenta.mp4
assim.mp4
ator.mp4
atual.mp4
autor.mp4
banda.mp4
bem.mp4
brasileiro.mp4
campanha.mp4
campo.mp4
capital.mp4
casa.mp4
casamento.mp4
categoria.mp4
centro.mp4
cerca.mp4
cinco.mp4
cinema.mp4
classe.mp4
clube.mp4
comunidade.mp4
conta.mp4
contra.mp4
contrato.mp4
controle.mp4
corpo.mp4
data.mp4
dentro.mp4
deu.mp4
deve.mp4
devido.mp4
dez.mp4
dezembro.mp4
dia.mp4
direito.mp4
disco.mp4
disso.mp4
distrito.mp4
dois.mp4
embora.mp4
empresa.mp4
equipe.mp4
escola.mp4
especial.mp4
esposa.mp4
estado.mp4
estudos.mp4
exemplo.mp4
faixa.mp4
fazer.mp4
fevereiro.mp4
fez.mp4
filme.mp4
fim.mp4
final.mp4
forma.mp4
forte.mp4
frente.mp4
futebol.mp4
geral.mp4
governo.mp4
grande.mp4
grupo.mp4
guerra.mp4
hoje.mp4
homem.mp4
idade.mp4
igreja.mp4
ilha.mp4
in.mp4
interior.mp4
internacional.mp4
jovem.mp4
julho

In [ ]:
# tensorboard = TensorBoard(log_dir="logs/train_cnnXrnn_baseline_mp")
# model.fit(X_mp, Y, epochs=1000,
#           # validation_data=(train_generator_mp),
#           # callbacks=[tensorboard, estop])
#           callbacks=[tensorboard, estop])
# #treino com o dataset com preprocess e mediapipe extractor


# for idx, test in enumerate(mp_data):
#   # x_test = test.reshape(-1, 32, 32, 1)
#   if files[np.argmax(model.predict(np.array([test]), verbose=0))] == files[idx]:
#     print(files[idx])
model.save('ariel-mp.keras')


In [ ]:
%tensorboard --logdir logs